In [6]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import utilsCM

from tqdm import tqdm

In [7]:
#Load word2sense
#already preprocessed in MATLAB so that Wrd2Sns and THINGs overlap --> we have IMAGES, LABELS and SENSES
pathtofile = '../code-00-preprocessdataset/'
Y_embeddings = pd.read_csv(pathtofile + "ThingsWrd2Sns.txt", sep=",")
Y_embeddings = Y_embeddings.values[:,1:Y_embeddings.shape[1]-1].astype(np.float)

In [8]:
datapath = '../../../data-04/'
nsample = 12
WrdThingsInfo = pd.read_csv(datapath + 'KeptTHINGSInfo_n' + str(nsample) +'.txt',sep=',',index_col = 0)

In [15]:
Y_embeddings_subset = Y_embeddings[WrdThingsInfo['old_index']]

In [16]:
len(Y_embeddings_subset)

312

In [17]:
layer =  {'conv_1','conv_5','fc_3'}

Sub = [1,2,3,4]


pretrained_val = True

datapath = '../../../data-00/'
savepath = '../../../data-05/'

In [18]:
objectROI = {'LOC','PFS'}
for ilayer in layer:
    predictor_variable = {}
    for iSub in Sub:        
            
        if not pretrained_val:
            filename = datapath +  "ROIpred_Sub" + str(iSub) + '_ObjectROI_' + ilayer + '_untrained_subset.npy'
        else:
            filename = datapath +  "ROIpred_Sub" + str(iSub) + '_ObjectROI_' + ilayer + '_subset.npy'

        if not os.path.isfile(filename): #if it's already saves
            for iROI in objectROI:
                if not pretrained_val:
                    thisSub = np.load(datapath +  "ROIpred_Sub" + str(iSub) + '_' + iROI + "_" + ilayer + '_untrained.npy')
                else:
                    thisSub = np.load(datapath +  "ROIpred_Sub" + str(iSub) + '_' + iROI + "_" + ilayer + '.npy')
            
                #load ROIpred as predictor variable
                if iSub is 1:
                    predictor_variable = thisSub[WrdThingsInfo['old_index']]
                else:
                    predictor_variable = np.append( predictor_variable , thisSub[WrdThingsInfo['old_index']], axis = 1)    
            np.save(filename,predictor_variable)

In [19]:
predictor_variable

{}